# Block 6 Exercise 1: Non-Linear Classification

## MNIST Data
We return to the MNIST data set on handwritten digits to compare non-linear classification algorithms ...   

In [0]:
#imports 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_openml

In [0]:
# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

In [3]:
#the full MNIST data set contains 70k samples of digits 0-9 as 28*28 gray scale images (represented as 784 dim vectors)
np.shape(X)

(70000, 784)

In [4]:
X.min()

0.0

In [5]:
#look at max/min value in the data
X.max()

255.0

### E1.1: Cross-Validation and Support Vector Machines
Train and optimize  C-SVM classifier on MNIST (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)
* use a RBF kernel
* use *random search* with cross-validation to find the best settings for *gamma* and *C* (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV)

In [0]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

svc = SVC(gamma='auto', kernel='rbf')

In [0]:
#erstmal normales svm trainieren mit score testen damit mans vergleichen kann
#nur parts der Daten nehmen wenn es immernoch lange dauert

In [0]:
distributions = dict(C=[0.01,0.1, 1, 10, 100], gamma=[1,0.1,0.01,0.001])

clf = RandomizedSearchCV(svc, distributions, n_jobs=8, n_iter=30, cv=5, random_state=0)


In [0]:
search= clf.fit(X, y)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 20 is smaller than n_iter=30. Running 20 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [0]:
search.best_params_

In [0]:
#dann nochmal mit den best params ausführen zum gegen test von anfang

### E1.2: Pipelines and simple Neural Networks
Split the MNIST data into  train- and test-sets and then train and evaluate a simple Multi Layer Perceptron (MLP) network. Since the non-linear activation functions of MLPs are sensitive to the scaling on the input (recall the *sigmoid* function), we need to scale all input values to [0,1] 

* combine all steps of your training in a SKL pipeline (https://scikit-learn.org/stable/modules/compose.html#pipeline)
* use a SKL-scaler to scale the data (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
* MLP Parameters: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier
    * use a *SGD* solver
    * use *tanh* as activation function
    * compare networks with 1, 2 and 3 layers, use different numbers of neurons per layer
    * adjust training parameters *alpha* (regularization) and *learning rate* - how sensitive is the model to these parameters?
    * Hint: do not change all parameters at the same time, split into several experiments
* How hard is it to find the best parameters? How many experiments would you need to find the best parameters?
    


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [0]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

clf = make_pipeline(StandardScaler(), MLPClassifier(solver='sgd', activation='tanh', hidden_layer_sizes=(100,), alpha=0.0001, learning_rate='constant', random_state=0))
clf.fit(X_train, y_train)

In [0]:
clf2 = make_pipeline(StandardScaler(), MLPClassifier(solver='sgd', activation='tanh', hidden_layer_sizes=(100,50,), alpha=0.0001, learning_rate='constant', random_state=0))
clf2.fit(X_train, y_train)

In [0]:
clf3 = make_pipeline(StandardScaler(), MLPClassifier(solver='sgd', activation='tanh', hidden_layer_sizes=(100,50,10,), alpha=0.0001, learning_rate='constant', random_state=0))
clf3.fit(X_train, y_train)

Die richtigen Parameter zu finden, braucht viel Zeit, da man immer wieder warten muss, bis man die Lösungen erhält und es viele Parameter gibt die man verschieden stark ändern kann.